This will be an initial formulation for my problem where the code will be rough. I will try to condense this notebook into a .py file for my final formulation. 


First steps will be fomulating the model.
See how many we can feasibly run on local machine. 
Create fake document with multiple locations and distances. 


In [1]:
%pip install gurobipy

   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
    --------------------------------------- 0.3/11.2 MB 5.2 MB/s eta 0:00:03
   - -------------------------------------- 0.5/11.2 MB 5.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.2 MB 5.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.2 MB 3.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.2 MB 5.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.2 MB 5.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.2 MB 5.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/11.2 MB 5.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/11.2 MB 5.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/11.2 MB 5.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.2 MB 5.3 MB/s eta 0:00:02
   ---------- ----------------------------- 3.0/11.2 MB 5.3 MB/s eta 0:00:02
   ---

In [3]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import Model,GRB,quicksum

First I will define sets and parameters these will most likely be put in CSVs for the real problem and I will read them in via pandas. 

In [7]:
# Sets
I = [1, 2, 3, 4, 5]  # TLAMM
J = [1, 2, 3, 4]  # Unit
K = [1, 2]  # Cost factors

# Parameters
c = {1: 1000, 2: 1500, 3: 1200, 4:400, 5:10000}  # Fixed costs for TLAMMs
d = {(i, j): 10 for i in I for j in J}  # Example distances
w = {1: 0.5, 2: 0.5}  # Weights for cost factors
f_kij = {(k, i, j): d[(i, j)] for k in K for i in I for j in J}  # Example costs
P = 500  # Penalty for extra TLAMM
L = 2  # Maximum number of TLAMM
k_cap = {1: 50, 2: 60, 3: 70, 4:50, 5:150}  #TLAMM capacities


In [9]:
# Model Implementation
model = Model("TLAMM Problem")

# Decision variables
x = model.addVars(I, vtype=GRB.BINARY, name="x")  # Binary for TLAMM opening
y = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="y")  # Continuous for assignments
M = model.addVar(vtype=GRB.CONTINUOUS, name="M")  # Excess TLAMMs

# Objective Function
model.setObjective(
    quicksum(c[i] * x[i] for i in I) +
    quicksum(quicksum(quicksum(w[k] * f_kij[k, i, j] for k in K) * y[i, j] for j in J) for i in I) +
    P * M,
    GRB.MINIMIZE
)

# Constraints
# 1. Demand satisfaction
model.addConstrs((quicksum(y[i, j] for i in I) == 1 for j in J), name="DemandSatisfaction")

# 2. Assignment only if TLAMM is open
model.addConstrs((y[i, j] <= x[i] for i in I for j in J), name="AssignmentValidity")

# 3. Limit on number of TLAMMs
model.addConstr(quicksum(x[i] for i in I) <= L, name="TLAMMLimit")

# 4. Capacity constraints
model.addConstrs((quicksum(y[i, j] for j in J) <= k_cap[i] * x[i] for i in I), name="Capacity")

# 5. Excess warehouses
model.addConstr(M >= quicksum(x[i] for i in I) - 1, name="ExcessLocation")
model.addConstr(M >= 0, name="NonNegativeExcess")

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Objective Value: {model.objVal}")
    for i in I:
        print(f"Warehouse {i} Open: {x[i].x}")
    for i in I:
        for j in J:
            if y[i, j].x > 0:
                print(f"Customer {j} served by TLAMM {i}: {y[i, j].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 32 rows, 26 columns and 97 nonzeros
Model fingerprint: 0xff77de4e
Variable types: 21 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 25 rows and 14 columns
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 22 nonzeros
Variable types: 6 continuous, 6 integer (6 binary)
Found heuristic solution: objective 12040.000000
Found heuristic solution: objective 10940.000000
Found heuristic solution: objective 3040.0000000

Root relaxation: objective 4.400000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    